In [ ]:
import numpy as np
import pandas as pd
import json


In [ ]:
def json_to_csv(filename):
    tweets = []
    with open(filename, 'r') as f:
        tweets = f.readlines()
    f.close()
    for i in range(len(tweets[:])):
        if type(tweets[i])==type('hi'):
            tweets[i] = json.loads(tweets[i])
    columns = ['in_reply_to_user_id', 'place', 'entities', 'created_at',
           'withheld_in_countries', 'display_text_range', 'in_reply_to_status_id',
           'favorite_count', 'geo', 'quoted_status_id', 'id_str',
           'withheld_copyright', 'possibly_sensitive', 'extended_entities',
           'retweet_count', 'full_text', 'quoted_status_permalink', 'id',
           'contributors', 'user', 'coordinates', 'quoted_status', 'retweeted',
           'favorited', 'quoted_status_id_str', 'withheld_scope',
           'in_reply_to_user_id_str', 'source', 'truncated', 'is_quote_status',
           'in_reply_to_status_id_str', 'in_reply_to_screen_name', 'lang']
    df = pd.DataFrame(tweets, columns=columns)
    return df

In [ ]:
df=json_to_csv('/content/tweets.json')
df.iloc[0]

In [ ]:
for i in range(len(df)):
    a=df.iloc[i]["full_text"]
    text=" ".join(filter(lambda x:x[0]=='@', a.split()))
    print(text)

In [ ]:
def preprocess(text):
    import re
    text = text.lower()
    text = re.sub(r'[^\w\s]', '', text)
    text = re.sub(" +", " ", text).strip()
    return text

In [ ]:
def all_texts(username):
  import re
  texts=[]
  df=get_tweets(username)
  for i in range(len(df)):
    a=df[i].lower()
    text=" ".join(filter(lambda x:ord(x[0])>96 and ord(x[0])<123, a.split()))
    text=preprocess(text)
    texts.append(text)
  return texts

In [ ]:
!pip install hatesonar
!pip install geocoder

     |████████████████████████████████| 890kB 6.2MB/s 
     |████████████████████████████████| 102kB 4.0MB/s 


In [ ]:
 from hatesonar import Sonar
 sonar=Sonar()

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.linear_model.logistic module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.linear_model. Anything that cannot be imported from sklearn.linear_model is now part of the private API.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator LogisticRegression from version 0.19.1 when using version 0.22.2.post1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator TfidfTransformer from version 0.19.1 when using version 0.22.2.post1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/base.py

In [ ]:
def hate_speech(username):
  hate_tweets=[]
  texts=all_texts(username)
  #texts.append("At least I'm not a nigger")
  
  for i  in range(len(texts)):
    t=texts[i]
    ans=sonar.ping(text=t)
    if ans['top_class'] =='hate_speech':
      hate_tweets.append(t)
  
  if len(hate_tweets)==0:
    return "No hate tweets recorded"
    
  return hate_tweets

In [ ]:
import tweepy
import geocoder


In [ ]:
auth = tweepy.AppAuthHandler('yQzsQvzdx4kOYqJiKC0i1pAfy','F6pcQU4hBJ0r6rCh5B2tlHxjDjVNgETZgeDH9oH2hdE2QBDkhm')

api = tweepy.API(auth)

In [ ]:
def trending(filename="available_locs_for_trend.json"):
    loc = "India"
    g = geocoder.osm(loc) # getting object that has location's latitude and longitude

    closest_loc = api.trends_closest(g.lat, g.lng)
    trends = api.trends_place(closest_loc[0]['woeid'])
    with open(filename,"w") as wp:
        wp.write(json.dumps(trends, indent=1))



In [ ]:
trending()

In [ ]:
def json_to_csv_trending(filename="available_locs_for_trend.json"):
    with open(filename, 'r') as j:
      contents = json.loads(j.read())
    d=contents[0]["trends"]
    df=pd.json_normalize(d)
    return df
    # #   columns = ["Name","URL","pc","query","tweet_volume"]
    #   df = pd.DataFrame(contents, columns=columns)
    # print(df.head())
    # return df

In [ ]:
def trending_graph(filename="available_locs_for_trend.json"):
  import csv
  import plotly.graph_objects as go
  import chart_studio.plotly as py
  import chart_studio
  import pandas as pd
  df=json_to_csv_trending(filename)
  df=df.drop(columns=["url","promoted_content","query"])
  df=df.dropna()
  fig = go.Figure()

  fig.add_trace(go.Bar(x=df['name'], y=df['tweet_volume']))
  username = 'sajag18089'
  api_key = 'HrTA3QFIdveYJof5KcKu'
  chart_studio.tools.set_credentials_file(username=username, api_key=api_key)
  fig.update_layout(template='ggplot2')
  return py.plot(fig, filename = 'Trending_Now', auto_open=True)

  

In [ ]:
trending_graph()

'https://plotly.com/~sajag18089/20/'

In [ ]:
def most_hashtags(username):
  import csv
  import plotly.graph_objects as go
  import chart_studio.plotly as py
  import chart_studio
  import pandas as pd

  arr=get_tweets(username)
  from collections import Counter
  all_hashtags=list()
  for i in range(len(arr)):
    a=arr[i]
    all_hashtags.extend(list(filter(lambda x:x[0]=='#', a.split())))
  for i in range(len(all_hashtags)):
    all_hashtags[i]=preprocess(all_hashtags[i])
  d=Counter(all_hashtags)
  header_added=False
  with open('temp.csv', 'w') as csv_file:  
    writer = csv.writer(csv_file)
    if not header_added:
          writer.writerow(['a','b'])
          header_added = True
    for key, value in d.items():
       writer.writerow([key, value])
  df = pd.read_csv('temp.csv')
  fig = go.Figure()

  fig.add_trace(go.Bar(x=df['a'], y=df['b']))

  username = 'sajag18089'
  api_key = 'HrTA3QFIdveYJof5KcKu'
  chart_studio.tools.set_credentials_file(username=username, api_key=api_key)
  fig.update_layout(template='ggplot2')
  return py.plot(fig, filename = 'Most_Hashtags', auto_open=True)



In [ ]:
import tweepy
  

  

def get_tweets(username,gettime=False):
        number_of_tweets=50
        tweets = api.user_timeline(screen_name=username)
  

        tw=[]
        dt=[] 
        tweets_for_csv = [tweet.text for tweet in tweets]
        date_time=[tweet.created_at for tweet in tweets]
        for j in tweets_for_csv:
            tw.append(j) 
        
        for j in date_time:
            dt.append(j)
  
        if gettime:
          return tw,dt
        else:
          return tw
  
  

In [ ]:
arr,dt=get_tweets('WasimJaffer14',True)
arr,dt

In [ ]:
hate_speech('WasimJaffer14')

'No hate tweets recorded'

In [ ]:
!pip install chart_studio

In [ ]:
print(most_hashtags('WasimJaffer14'))

https://plotly.com/~sajag18089/3/


In [ ]:
#INDvsENG #INDvsENG…

In [ ]:
def posting_pattern(Username):
  import csv
  import plotly.graph_objects as go
  import chart_studio.plotly as py
  import chart_studio
  import pandas as pd
  import datetime
  arr,dt=get_tweets(Username,True)
  for i in range(len(dt)):
    dt[i] = dt[i].replace(hour=0, minute=0, second=0, microsecond=0)
  dt.sort()
  cnt={}
  now=dt[0]
  while now<=dt[-1]:
    for i in range(len(dt)):
      if dt[i]==now:
        if dt[i] in cnt:
          cnt[dt[i]]+=1
        else:
          cnt[dt[i]]=1
    now+=datetime.timedelta(days=1)
  header_added=False
  with open('temp.csv', 'w') as csv_file:  
    writer = csv.writer(csv_file)
    if not header_added:
          writer.writerow(['a','b'])
          header_added = True
    for key, value in cnt.items():
       writer.writerow([key, value])
  df = pd.read_csv('temp.csv')
  fig = go.Figure()

  fig.add_trace(go.Bar(x=df['a'], y=df['b']))

  username = 'sajag18089'
  api_key = 'HrTA3QFIdveYJof5KcKu'
  chart_studio.tools.set_credentials_file(username=username, api_key=api_key)
  return py.plot(fig, filename = "Posting_Patterns", auto_open=True)



In [ ]:
print(posting_pattern('WasimJaffer14'))

https://plotly.com/~sajag18089/16/
